<a href="https://colab.research.google.com/github/AkshayWaghela/FCC/blob/notebooks/AW_FCC_sms_text_classificationNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#?pd.read_table

In [ ]:
d1 = pd.read_table(train_file_path,names=['C','T'])
d1.head()

In [ ]:
dt1 = pd.read_table(test_file_path,names=['C','T'])
dt1.head()

In [ ]:
# Check for basic info: rows, columns, data types, missing values
df = d1
df.info()

# Check for any missing values
print(df.isnull().sum())

# Check basic statistics of the data
df.describe(include='all')

# View the distribution of categories in 'C' column
print(df['C'].value_counts())


In [ ]:
d1.columns

In [ ]:
df = d1.drop_duplicates(ignore_index=True)
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.C.value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
d1.C.value_counts(normalize=True).plot(kind='bar')

In [ ]:
sent_set = []
label_set = []
for i in range(0,df.shape[0],1):
    sent_set.append(df['T'][i])
    label_set.append(df['C'][i])

In [ ]:
d1.to_excel('SMS_DATA.xlsx')

In [ ]:
print("Stage I. Preliminary actions. Preparing of needed sets\n")

sentences_new_set = []
labels_new_set = []
for i in range(0, d1.shape[0], 1):
    sentences_new_set.append(d1['T'][i])
    labels_new_set.append(d1['C'][i])

In [ ]:
size_vocabulary = 1000    # You can choose the size of vocabulary yourself in the range [500 .. 1500] (it should be divisible by 500)
embedding_dimension = 64  # You can choose the size of dimention yourself in the range [32 .. 256] (it should be divisible by 32)
trunc_type = 'post'
padding_type = 'post'
threshold = 0.5           # You can choose the size of threshold yourself in the range [0 .. 1]
oov_token = "<OOV>"
test_size, valid_size = 0.05, 0.2
num_epochs = 20           # You can choose the number of epochs yourself in the range [20 .. 50] (it should be divisible by 5)
drop_level = 0.3          # You can choose the size of drop level yourself in the range [0 .. 1]

In [ ]:
train_size = int(d1.shape[0] * (1 - test_size - valid_size))
valid_bound = int(d1.shape[0] * (1 - valid_size))

train_sentences = sentences_new_set[0 : train_size]
valid_sentences = sentences_new_set[train_size : valid_bound]
test_sentences = sentences_new_set[valid_bound : ]

train_labels_str = labels_new_set[0 : train_size]
valid_labels_str = labels_new_set[train_size : valid_bound]
test_labels_str = labels_new_set[valid_bound : ]

In [ ]:
print("Stage II. Labels transformations\n")

train_labels = [0] * len(train_labels_str)
for ind, item in enumerate(train_labels_str):
    if item == 'ham':
        train_labels[ind] = 1
    else:
        train_labels[ind] = 0

valid_labels = [0] * len(valid_labels_str)
for ind, item in enumerate(valid_labels_str):
    if item == 'ham':
        valid_labels[ind] = 1
    else:
        valid_labels[ind] = 0

test_labels = [0] * len(test_labels_str)
for ind, item in enumerate(test_labels_str):
    if item == 'ham':
        test_labels[ind] = 1
    else:
        test_labels[ind] = 0

train_labels = np.array(train_labels)
valid_labels = np.array(valid_labels)
test_labels = np.array(test_labels)
#print(test_labels)

In [ ]:
import nltk, re, collections, pickle, os # nltk - Natural Language Toolkit
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# %matplotlib inline
plt.rcParams['figure.figsize'] = (15, 5)
plt.style.use('ggplot')
seed = 42

import warnings
warnings.filterwarnings(action = "ignore")
warnings.simplefilter(action = 'ignore', category = Warning)
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('punkt')

In [ ]:
print("Stage III. Tokenization\n")

tokenizer = Tokenizer(num_words = size_vocabulary,
                      oov_token = oov_token,
                      lower = False)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

In [ ]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
size_voc = len(word_index) + 1
max_len = max([len(i) for i in train_sequences])
train_set = pad_sequences(train_sequences,
                                padding = padding_type,
                                maxlen = max_len,
                                truncating = trunc_type)

valid_sequences = tokenizer.texts_to_sequences(valid_sentences)
valid_set = pad_sequences(valid_sequences,
                               padding = padding_type,
                               maxlen = max_len,
                               truncating = trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_set = pad_sequences(test_sequences,
                               padding = padding_type,
                               maxlen = max_len,
                               truncating = trunc_type)

In [ ]:
print("Stage IV. Model building\n")

model = Sequential([
    Embedding(size_voc, embedding_dimension, input_length = max_len),
    Bidirectional(LSTM(100)),
    Dropout(drop_level),
    Dense(20, activation = 'relu'),
    Dropout(drop_level),
    Dense(1, activation = 'sigmoid')
])

In [ ]:
print("Stage V. Model compiling & fitting\n")
optim = Adam(learning_rate = 0.001)

model.compile(loss = 'binary_crossentropy',
              optimizer = optim,
              metrics = ['accuracy'])
model.summary()

In [ ]:
?model.fit()

In [ ]:
len(train_set)

In [ ]:
history = model.fit(train_set,
                    train_labels,
                    epochs = 5,validation_data = (valid_set, valid_labels),
                    verbose = 1)

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'accuracy' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'accuracy' in s and 'val' in s]

    plt.figure(figsize = (12, 5), dpi = 100)
    COLOR = 'gray'

    plt.rc('legend', fontsize = 14)   # legend fontsize
    plt.rc('figure', titlesize = 12)  # fontsize of the figure title

    if len(loss_list) == 0:
        print('Loss is missing in history')
        return

    ## As loss always exists
    epochs = range(1, len(history.history[loss_list[0]]) + 1)

    ## Loss
    plt.subplot(1, 2, 1)
    plt.subplots_adjust(wspace = 2, hspace = 2)
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.titlecolor'] = 'black'
    plt.rcParams['axes.labelcolor'] = COLOR
    plt.rcParams['xtick.color'] = COLOR
    plt.rcParams['ytick.color'] = COLOR
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b-o',
                 label = 'Train (' + str(str(format(history.history[l][-1],'.4f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g',
                 label = 'Valid (' + str(str(format(history.history[l][-1],'.4f'))+')'))

    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend(facecolor = 'gray', loc = 'best')
    plt.grid(True)
    plt.tight_layout()

    ## Accuracy
    plt.subplot(1, 2, 2)
    plt.subplots_adjust(wspace = 2, hspace = 2)
    plt.rcParams['text.color'] = 'black'
    plt.rcParams['axes.titlecolor'] = 'black'
    plt.rcParams['axes.labelcolor'] = COLOR
    plt.rcParams['xtick.color'] = COLOR
    plt.rcParams['ytick.color'] = COLOR
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b-o',
                 label = 'Train (' + str(format(history.history[l][-1],'.4f'))+')')
    for l in val_acc_list:
        plt.plot(epochs, history.history[l], 'g',
                 label = 'Valid (' + str(format(history.history[l][-1],'.4f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend(facecolor = 'gray', loc = 'best')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_conf_matr(conf_matr, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.winter):
  """
  Citation
  ---------
  http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

  """
  import itertools

  accuracy = np.trace(conf_matr) / np.sum(conf_matr).astype('float')
  sns.set(font_scale = 1.4)

  plt.figure(figsize = (12, 8))
  plt.imshow(conf_matr, interpolation = 'nearest', cmap = cmap)
  title = '\n' + title + '\n'
  plt.title(title)
  plt.colorbar()

  if classes is not None:
      tick_marks = np.arange(len(classes))
      plt.xticks(tick_marks, classes, rotation = 45)
      plt.yticks(tick_marks, classes)



  thresh = conf_matr.max() / 1.5 if normalize else conf_matr.max() / 2
  for i, j in itertools.product(range(conf_matr.shape[0]), range(conf_matr.shape[1])):
      if normalize:
          plt.text(j, i, "{:0.2f}%".format(conf_matr[i, j] * 100),
                    horizontalalignment = "center",
                    fontweight = 'bold',
                    color = "white" if conf_matr[i, j] > thresh else "black")
      else:
          plt.text(j, i, "{:,}".format(conf_matr[i, j]),
                    horizontalalignment = "center",
                    fontweight = 'bold',
                    color = "white" if conf_matr[i, j] > thresh else "black")
  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label\n\nAccuracy = {:0.2f}%; Error = {:0.2f}%'.format(accuracy * 100, (1 - accuracy) * 100))
  plt.show()


def plot_words(set, number):
  words_counter = collections.Counter([word for sentence in set for word in sentence.split()]) # finding words along with count
  most_counted = words_counter.most_common(number)
  most_count = pd.DataFrame(most_counted, columns = ["Words", "Amount"]).sort_values(by = "Amount") # sorted data frame
  most_count.plot.barh(x = "Words",
                       y = "Amount",
                       color = "blue",
                       figsize = (10, 15))
  for i, v in enumerate(most_count["Amount"]):
    plt.text(v, i,
             " " + str(v),
             color = 'black',
             va = 'center',
             fontweight = 'bold')

def word_cloud(tag):
  df_words_nl = ' '.join(list(df_spam[df_spam['feature'] == tag]['message']))
  df_wc_nl = WordCloud(width = 600, height = 512).generate(df_words_nl)
  plt.figure(figsize = (13, 9), facecolor = 'k')
  plt.imshow(df_wc_nl)
  plt.axis('off')
  plt.tight_layout(pad = 1)
  plt.show()

In [ ]:
print("Stage VI. Results visualization\n")
plot_history(history)

In [ ]:
model_score = model.evaluate(test_set, test_labels, batch_size = embedding_dimension, verbose = 1)
print(f"Test accuracy: {model_score[1] * 100:0.2f}% \t\t Test error: {model_score[0]:0.4f}")

In [ ]:
y_pred_bLSTM = model.predict(test_set)

y_prediction = [0] * y_pred_bLSTM.shape[0]
for ind, item in enumerate(y_pred_bLSTM):
    if item > threshold:
        y_prediction[ind] = 1
    else:
        y_prediction[ind] = 0

conf_m_bLSTM = confusion_matrix(test_labels, y_prediction)
class_rep_bLSTM = classification_report(test_labels, y_prediction)
print('\t\t\tClassification report:\n\n', class_rep_bLSTM, '\n')
plot_conf_matr(conf_m_bLSTM, classes = ['Spam','Ham'], normalize = False, title = 'Confusion matrix for bLSTM')

In [ ]:
# You can change this message (as any short sentence) yourself
message_example = ["Darling, please give me a cup of tea"]

message_example_tp = pad_sequences(tokenizer.texts_to_sequences(message_example),
                                   maxlen = max_len,
                                   padding = padding_type,
                                   truncating = trunc_type)

pred = float(model.predict(message_example_tp))
if (pred > threshold):
    print ("This message is a real text")
else:
    print("This message is a spam message")

In [ ]:
# function to predict messages based on modelva
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # You can change this message (as any short sentence) yourself
  message_example = [pred_text]

  message_example_tp = pad_sequences(tokenizer.texts_to_sequences(message_example),
                                   maxlen = max_len,
                                   padding = padding_type,
                                   truncating = trunc_type)

  pred = float(model.predict(message_example_tp))
  prediction = ''
  if (pred > threshold):
      print ("ham")
      prediction = 'ham'
  else:
      prediction = 'spam'
      print("spam")

  return ([pred,prediction])

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction,"---",ans)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


In [ ]:
plt.figure(figsize=(6,4))
sns.countplot(x=df['C'], palette='coolwarm')
plt.title("Distribution of Spam vs. Ham Messages")
plt.show()


In [ ]:
df['length'] = df['T'].apply(len)
plt.figure(figsize=(10,5))
sns.histplot(df[df['C']=='ham']['length'], bins=50, color='blue', label='Ham', kde=True)
sns.histplot(df[df['C']=='spam']['length'], bins=50, color='red', label='Spam', kde=True)
plt.legend()
plt.title("Message Length Distribution")
plt.show()


In [ ]:
from wordcloud import WordCloud

spam_words = ' '.join(df[df['C']=='spam']['T'])
ham_words = ' '.join(df[df['C']=='ham']['T'])

plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.imshow(WordCloud(width=500, height=300, background_color="black").generate(spam_words))
plt.title("Spam Word Cloud")
plt.axis("off")

plt.subplot(1,2,2)
plt.imshow(WordCloud(width=500, height=300, background_color="black").generate(ham_words))
plt.title("Ham Word Cloud")
plt.axis("off")

plt.show()


In [ ]:
from collections import Counter

words = ' '.join(df['T']).split()
word_freq = Counter(words)
common_words = pd.DataFrame(word_freq.most_common(10), columns=['Word', 'Count'])

plt.figure(figsize=(10,5))
sns.barplot(x='Count', y='Word', data=common_words, palette='viridis')
plt.title("Top 10 Most Common Words")
plt.show()


In [ ]:
print("Average Message Length (Spam):", df[df['C']=='spam']['length'].mean())
print("Average Message Length (Ham):", df[df['C']=='ham']['length'].mean())


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_labels, y_prediction)

plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['Spam','Ham'], yticklabels=['Spam','Ham'])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

print("Classification Report:\n")
print(classification_report(test_labels, y_prediction, target_names=['Spam','Ham']))


In [ ]:
misclassified_indices = np.where(test_labels != y_prediction)[0]

for idx in misclassified_indices[:5]:  # Show first 5 misclassified messages
    print(f"Actual: {'Ham' if test_labels[idx] == 1 else 'Spam'}, Predicted: {'Ham' if y_prediction[idx] == 1 else 'Spam'}")
    print(f"Message: {test_sentences[idx]}\n")


In [ ]:
import numpy as np
import scipy.stats as stats

# Calculate lengths of messages
df['length'] = df['T'].apply(len)

# Extract spam & ham lengths
spam_lengths = df[df['C'] == 'spam']['length']
ham_lengths = df[df['C'] == 'ham']['length']

# Summary statistics
print(f"Spam Mean Length: {np.mean(spam_lengths):.2f}, Std Dev: {np.std(spam_lengths):.2f}")
print(f"Ham Mean Length: {np.mean(ham_lengths):.2f}, Std Dev: {np.std(ham_lengths):.2f}")


In [ ]:
levene_test = stats.levene(spam_lengths, ham_lengths)
print(f"Levene's Test p-value: {levene_test.pvalue:.5f}")

if levene_test.pvalue < 0.05:
    print("Variance is significantly different.")
else:
    print("Variances are similar.")


In [ ]:
t_test = stats.ttest_ind(spam_lengths, ham_lengths, equal_var=False)  # Welch’s t-test
print(f"T-test Statistic: {t_test.statistic:.2f}, p-value: {t_test.pvalue:.5f}")

if t_test.pvalue < 0.05:
    print("There is a significant difference in message length between spam and ham.")
else:
    print("No significant difference in length.")


In [ ]:
def cohens_d(x, y):
    return (np.mean(x) - np.mean(y)) / np.sqrt((np.std(x) ** 2 + np.std(y) ** 2) / 2)

effect_size = cohens_d(spam_lengths, ham_lengths)
print(f"Cohen's d: {effect_size:.2f}")

if effect_size >= 0.8:
    print("Large difference in message lengths between spam and ham.")
elif effect_size >= 0.5:
    print("Moderate difference in message lengths.")
else:
    print("Small difference in message lengths.")


In [ ]:
from sklearn.metrics import roc_auc_score

labels = (df['C'] == 'spam').astype(int)  # Convert 'spam' = 1, 'ham' = 0
auc = roc_auc_score(labels, df['length'])
print(f"AUC Score for Length-Based Classification: {auc:.3f}")

if auc > 0.8:
    print("Message length is a strong predictor of spam.")
elif auc > 0.6:
    print("Message length provides some predictive power.")
else:
    print("Message length alone is not a reliable predictor of spam.")


In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set(style="whitegrid")

# Create figure
plt.figure(figsize=(12, 5))

# KDE Plot
sns.kdeplot(spam_lengths, label="Spam", fill=True, color="red", alpha=0.5)
sns.kdeplot(ham_lengths, label="Ham", fill=True, color="blue", alpha=0.5)

# Titles and labels
plt.title("Distribution of Message Lengths", fontsize=14)
plt.xlabel("Message Length", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.legend()

# Show KDE plot
plt.show()

# Boxplot for comparison
plt.figure(figsize=(8, 5))
sns.boxplot(x=df['C'], y=df['length'], palette={"spam": "red", "ham": "blue"})

# Annotate Cohen’s d
plt.text(0.5, max(df['length']) * 0.95, f"Cohen's d = {effect_size:.2f}", ha='center', fontsize=12)

plt.title("Boxplot of Message Lengths")
plt.xlabel("Message Category")
plt.ylabel("Message Length")
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm

# Compute message lengths
df['length'] = df['T'].apply(len)

# Separate spam and ham lengths
spam_lengths = df[df['C'] == 'spam']['length']
ham_lengths = df[df['C'] == 'ham']['length']

# Fit normal distributions
spam_mean, spam_std = np.mean(spam_lengths), np.std(spam_lengths)
ham_mean, ham_std = np.mean(ham_lengths), np.std(ham_lengths)

# Prior probabilities
P_spam = len(spam_lengths) / len(df)
P_ham = len(ham_lengths) / len(df)

# Define function to compute P(Spam | Length)
def spam_likelihood(length):
    P_length_given_spam = norm.pdf(length, spam_mean, spam_std)
    P_length_given_ham = norm.pdf(length, ham_mean, ham_std)

    P_length = P_length_given_spam * P_spam + P_length_given_ham * P_ham

    return (P_length_given_spam * P_spam) / P_length

# Example: Likelihood for a message of 100 characters
length_example = 100
spam_prob = spam_likelihood(length_example) * 100

print(f"Likelihood of a message being spam given length {length_example}: {spam_prob:.2f}%")


In [ ]:
spam_likelihood(130)*100

In [ ]:
from collections import Counter

# Tokenize words
spam_words = " ".join(df[df['C'] == 'spam']['T']).split()
ham_words = " ".join(df[df['C'] == 'ham']['T']).split()

# Word frequencies
spam_word_freq = Counter(spam_words)
ham_word_freq = Counter(ham_words)

# Compute likelihood of spam for a given word
def spam_word_likelihood(word):
    P_word_given_spam = spam_word_freq[word] / sum(spam_word_freq.values())
    P_word_given_ham = ham_word_freq[word] / sum(ham_word_freq.values())

    P_word = P_word_given_spam * P_spam + P_word_given_ham * P_ham

    return (P_word_given_spam * P_spam) / P_word if P_word > 0 else 0

# Example: Compute likelihood for the word "free"
word_example = "free"
word_spam_prob = spam_word_likelihood(word_example) * 100

print(f"Likelihood of a message being spam given it contains '{word_example}': {word_spam_prob:.2f}%")


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from collections import Counter

# Compute message lengths
df['length'] = df['T'].apply(len)

# Separate spam and ham lengths
spam_lengths = df[df['C'] == 'spam']['length']
ham_lengths = df[df['C'] == 'ham']['length']

# Fit normal distributions
spam_mean, spam_std = np.mean(spam_lengths), np.std(spam_lengths)
ham_mean, ham_std = np.mean(ham_lengths), np.std(ham_lengths)

# Prior probabilities
P_spam = len(spam_lengths) / len(df)
P_ham = len(ham_lengths) / len(df)

# Compute spam probability for different lengths
length_range = np.arange(10, 300, 5)
spam_probs = [norm.pdf(l, spam_mean, spam_std) * P_spam /
              (norm.pdf(l, spam_mean, spam_std) * P_spam + norm.pdf(l, ham_mean, ham_std) * P_ham)
              for l in length_range]

# Plot spam probability vs. length
plt.figure(figsize=(8, 5))
sns.lineplot(x=length_range, y=spam_probs, color='red')
plt.xlabel("Message Length")
plt.ylabel("P(Spam | Length)")
plt.title("Spam Probability Based on Message Length")
plt.grid()
plt.show()

# ---- Word-Based Probability ----
spam_words = " ".join(df[df['C'] == 'spam']['T']).split()
ham_words = " ".join(df[df['C'] == 'ham']['T']).split()

spam_word_freq = Counter(spam_words)
ham_word_freq = Counter(ham_words)

# Compute likelihood for common words
common_words = ["free", "win", "offer", "urgent", "call", "meeting", "hello"]
word_spam_probs = [(w, spam_word_freq[w] / sum(spam_word_freq.values()) * P_spam /
                    ((spam_word_freq[w] / sum(spam_word_freq.values())) * P_spam +
                     (ham_word_freq[w] / sum(ham_word_freq.values())) * P_ham))
                   if w in spam_word_freq else (w, 0) for w in common_words]

# Convert to dictionary for plotting
word_dict = {w: p for w, p in word_spam_probs}

# Plot spam probability for words
plt.figure(figsize=(8, 5))
sns.barplot(x=list(word_dict.keys()), y=list(word_dict.values()), color='blue')
plt.xlabel("Words")
plt.ylabel("P(Spam | Word)")
plt.title("Spam Probability Based on Word Usage")
plt.ylim(0, 1)
plt.grid()
plt.show()
